<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=074cad968fed0d5931c525770e89425a5ec22229fcde0aa777fd51a104d0c870
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-29 10:28:42
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -45.82 K (-0.32%)
Current PnL: -26.33 L (-17.01%)
CY Booked + Current PnL: -12.04 L (-7.78%)
-------------------
Total profit:  1.56 L
Total loss:  -27.88 L
-------------------
Total Booked + Current PnL: 14.94 L (11.69%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.15%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.96 L (64.64%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.52,-19.18,23.85,0.09,19424.0,-19334.0,81443.0,89.11,39.0,M-SC,2.92,253.0,-1.00,0.58,7.00,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.67,8.77,10.04,19.69,20939.0,16818.0,208560.0,-1.91,55.0,X-MC,2.40,78.0,0.80,1.49,16.16,XY25,NTT,AC
52,NESTLEIND,1377.00,0.79,13.08,7.35,21.40,22061.0,34726.0,300152.0,4.93,66.0,X-LC,5.54,12.0,1.57,2.14,20.24,XY25,NTT,FMCG
50,MASFIN,398.61,-0.65,-3.92,27.03,22.05,25446.0,-3840.0,94140.0,-17.34,48.0,H-SC,6.54,164.0,-0.15,0.67,36.84,XR,ATH,FINANCE
43,ITC,452.00,-0.28,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,48.0,X-LC,2.04,5.0,-0.10,1.69,3.94,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,4.62,-37.55,116.63,35.27,82698.0,-42643.0,70906.0,-51.71,56.0,L-SC,8.75,270.0,-0.52,0.51,73.54,XR,NTT,HOTELS
57,RAJOOENG,143.10,3.04,-37.89,124.79,39.61,79441.0,-38840.0,63660.0,-57.95,36.0,H-SC,31.75,135.0,-0.49,0.45,3.04,AR,ATH,MISC
44,JCHAC,2282.00,1.47,-34.82,53.40,-0.01,44485.0,-44499.0,83306.0,16943.48,42.0,M-SC,12.53,233.0,-1.00,0.59,6.62,OX40N,NTT,AC
81,VBL,671.64,0.83,-1.66,37.96,35.67,117908.0,-5232.0,310610.0,-13.42,60.0,X-LC,3.20,4.0,-0.04,2.22,11.97,X40N,ATH,FMCG
52,NESTLEIND,1377.00,0.79,13.08,7.35,21.40,22061.0,34726.0,300152.0,4.93,66.0,X-LC,5.54,12.0,1.57,2.14,20.24,XY25,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
25,FINCABLES,1641.55,-2.06,-4.83,118.47,107.91,166465.0,-7132.0,140512.0,-22.58,45.0,M-SC,11.64,216.0,-0.04,1.00,4.77,OX40N,ATH,CABLES
69,SURYODAY,216.00,-2.01,-19.90,53.61,23.03,76894.0,-35639.0,143432.0,55.70,53.0,H-SC,6.87,167.0,-0.46,1.02,42.31,XR,NTT,BANKS
48,KPIGREEN,730.66,-1.94,-17.88,78.95,46.95,81233.0,-22405.0,102892.0,-24.08,39.0,H-SC,10.03,154.0,-0.28,0.73,28.90,MH,ATH,POWER
47,KANSAINER,340.00,-1.71,-22.12,45.70,13.47,95977.0,-59652.0,210015.0,-67.92,53.0,H-SC,6.85,158.0,-0.62,1.50,6.79,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-1.34,-0.15,81.94,81.67,76756.0,-137.0,93673.0,8438.89,69.0,L-SC,15.03,271.0,-0.0,0.67,81.4,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.06,-4.83,118.47,107.91,166465.0,-7132.0,140512.0,-22.58,45.0,M-SC,11.64,216.0,-0.04,1.00,4.77,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,-0.52,-19.18,23.85,0.09,19424.0,-19334.0,81443.0,89.11,39.0,M-SC,2.92,253.0,-1.00,0.58,7.00,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.27,-19.69,24.54,0.02,34404.0,-34365.0,140194.0,113.73,34.0,M-SC,9.83,234.0,-1.00,1.00,21.94,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-1.71,-22.12,45.70,13.47,95977.0,-59652.0,210015.0,-67.92,53.0,H-SC,6.85,158.0,-0.62,1.50,6.79,XY24,NTT,PAINTS
66,SIS,528.00,-1.12,-22.92,57.33,21.26,49062.0,-25454.0,85578.0,2020.73,55.0,H-SC,4.77,166.0,-0.52,0.61,15.62,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.0,9.23,57.71,72.27,106813.0,15637.0,185085.0,-6.08,63.0,M-LC,4.47,99.0,0.15,1.32,15.97,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.00,9.23,57.71,72.27,106813.0,15637.0,185085.0,-6.08,63.0,M-LC,4.47,99.0,0.15,1.32,15.97,XR,NTT,IT
6,ASIANTILES,137.00,-1.34,-0.15,81.94,81.67,76756.0,-137.0,93673.0,8438.89,69.0,L-SC,15.03,271.0,-0.00,0.67,81.40,XR,NTT,CERAMICS
25,FINCABLES,1641.55,-2.06,-4.83,118.47,107.91,166465.0,-7132.0,140512.0,-22.58,45.0,M-SC,11.64,216.0,-0.04,1.00,4.77,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.50,-5.24,118.15,106.72,138089.0,-6460.0,116876.0,-53.11,37.0,H-SC,5.04,139.0,-0.05,0.83,16.92,AR,ATH,MISC
1,ABB,7934.00,0.04,-4.87,53.02,45.57,131956.0,-12739.0,248880.0,-38.86,51.0,H-MC,4.54,120.0,-0.10,1.78,5.90,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.65,-19.04,92.04,55.47,181281.0,-46327.0,196959.0,-28.25,27.0,X-MC,16.53,64.0,-0.26,1.41,0.92,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.85,-18.61,54.89,26.07,87220.0,-36322.0,158899.0,-60.43,27.0,X-MC,13.93,56.0,-0.42,1.13,0.00,X40N,ATH,IT
20,DABUR,735.00,-0.06,-4.61,50.65,43.70,119854.0,-11436.0,236632.0,-12.53,30.0,X-MC,2.06,72.0,-0.10,1.69,9.75,XY24,BTT,FMCG
8,AWL,485.00,-0.40,-25.31,105.51,53.50,237798.0,-76371.0,225380.0,-63.88,30.0,X-MC,6.83,58.0,-0.32,1.61,0.11,XY24,NTT,FMCG
34,HONAUT,58357.33,0.12,-18.84,76.60,43.32,101250.0,-30692.0,132180.0,-30.28,30.0,X-SC,6.59,90.0,-0.30,0.94,1.49,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-0.27,-3.69,25.79,21.14,56465.0,-8396.0,218943.0,-10.57,44.0,X-MC,1.24,74.0,-0.15,1.56,21.59,XY24,NTT,PAINTS
78,UNITDSPR,1644.0,-0.52,4.39,15.69,20.77,38348.0,10268.0,244412.0,-4.74,49.0,X-MC,1.70,62.0,0.27,1.75,11.61,X40N,NTT,BREWERIES
15,CAMS,950.0,-0.71,-2.58,27.77,24.48,68135.0,-6501.0,245355.0,-81.32,42.0,X-SC,1.87,86.0,-0.10,1.75,19.96,X40N,NTT,MISC
65,SIEMENS,4671.5,-0.39,-17.01,51.24,25.51,79135.0,-31655.0,154440.0,-5.19,41.0,H-LC,1.99,49.0,-0.40,1.10,13.81,AR,ATH,ELECTRICAL
43,ITC,452.0,-0.28,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,48.0,X-LC,2.04,5.0,-0.10,1.69,3.94,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.08,-27.13,125.27,64.15,217206.0,-64561.0,173390.0,-57.19,32.0,X-SC,4.78,82.0,-0.30,1.24,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.45,-39.57,123.73,35.20,96211.0,-50911.0,77759.0,1.95,32.0,X-SC,17.27,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.85,-18.61,54.89,26.07,87220.0,-36322.0,158899.0,-60.43,27.0,X-MC,13.93,56.0,-0.42,1.13,0.00,X40N,ATH,IT
19,COLPAL,3726.84,-0.63,-21.24,79.68,41.51,165272.0,-55945.0,207420.0,-6.31,34.0,X-MC,8.15,61.0,-0.34,1.48,0.00,XY25,ATH,FMCG
8,AWL,485.00,-0.40,-25.31,105.51,53.50,237798.0,-76371.0,225380.0,-63.88,30.0,X-MC,6.83,58.0,-0.32,1.61,0.11,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.26,-28.66,100.22,42.84,46480.0,-18628.0,46378.0,-53.17,48.0,X-SC,32.98,83.0,-0.40,0.33,4.92,XY24,NTT,MISC
59,RELAXO,1176.00,-0.63,-48.11,193.52,52.31,145763.0,-69838.0,75322.0,-44.73,40.0,X-SC,7.31,91.0,-0.48,0.54,0.82,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.45,-39.57,123.73,35.20,96211.0,-50911.0,77759.0,1.95,32.0,X-SC,17.27,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.47,-3.34,28.36,24.08,34802.0,-4234.0,122716.0,-11.10,41.0,X-SC,6.97,89.0,-0.12,0.88,15.63,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.12,-18.84,76.60,43.32,101250.0,-30692.0,132180.0,-30.28,30.0,X-SC,6.59,90.0,-0.30,0.94,1.49,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.61,-10.36,34.66,20.71,117511.0,-39197.0,339040.0,-22.65,58.0,X-LC,3.01,1.0,-0.33,2.42,13.29,X40,ATH,IT
41,INFY,1972.00,-0.02,12.72,19.10,34.24,67363.0,39797.0,352685.0,-12.06,65.0,X-LC,6.77,2.0,0.59,2.52,22.06,X40,NTT,IT
75,TMPV,600.00,0.47,-26.11,66.44,22.98,110896.0,-58986.0,166912.0,-23.08,51.0,X-LC,6.10,3.0,-0.53,1.19,4.98,XY24,NTT,AUTO
81,VBL,671.64,0.83,-1.66,37.96,35.67,117908.0,-5232.0,310610.0,-13.42,60.0,X-LC,3.20,4.0,-0.04,2.22,11.97,X40N,ATH,FMCG
43,ITC,452.00,-0.28,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,48.0,X-LC,2.04,5.0,-0.10,1.69,3.94,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.02,-37.25,111.76,32.89,54148.0,-28756.0,48450.0,-698.49,55.0,L-MC,6.83,259.0,-0.53,0.35,33.43,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
50,MASFIN,398.61,-0.65,-3.92,27.03,22.05,25446.0,-3840.0,94140.0,-17.34,48.0,H-SC,6.54,164.0,-0.15,0.67,36.84,XR,ATH,FINANCE
6,ASIANTILES,137.00,-1.34,-0.15,81.94,81.67,76756.0,-137.0,93673.0,8438.89,69.0,L-SC,15.03,271.0,-0.00,0.67,81.40,XR,NTT,CERAMICS
13,BSOFT,831.70,-0.48,-19.03,86.23,50.79,98201.0,-26770.0,113883.0,3.98,66.0,H-SC,10.09,151.0,-0.27,0.81,32.60,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-1.34,-0.15,81.94,81.67,76756.0,-137.0,93673.0,8438.89,69.0,L-SC,15.03,271.0,-0.00,0.67,81.40,XR,NTT,CERAMICS
13,BSOFT,831.7,-0.48,-19.03,86.23,50.79,98201.0,-26770.0,113883.0,3.98,66.0,H-SC,10.09,151.0,-0.27,0.81,32.60,XR,ATH,IT
74,TITAGARH,1548.0,0.47,-15.08,71.62,45.74,147291.0,-36521.0,205656.0,13.22,70.0,H-SC,5.85,174.0,-0.25,1.47,33.76,XY24,NTT,ENGINEERING
56,RAJESHEXPO,518.0,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
84,WIPRO,420.0,0.00,9.23,57.71,72.27,106813.0,15637.0,185085.0,-6.08,63.0,M-LC,4.47,99.0,0.15,1.32,15.97,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.47
1,25,44.95
2,50,76.60


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.91
MC    30.77
LC    24.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.86
X40      23.12
X40N     12.88
XR        9.86
XY25      9.19
AR        8.82
OX40N     7.52
SR        1.00
MH        0.73
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.18
H-SC    23.81
X-LC    20.82
M-SC    11.49
X-SC     8.05
H-MC     4.85
L-SC     1.56
M-MC     1.39
M-LC     1.32
H-LC     1.10
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-7.16,41.62
IT,12.99,-15.42,73.45
FINANCE,9.62,-18.23,69.65
MISC,7.14,-29.57,82.57
ELECTRICAL,5.95,-12.58,53.82
PAINTS,5.79,-13.71,30.60
INSURANCE,4.70,-3.05,38.32
PHARMA,4.07,-3.05,35.49
AUTO,2.84,-31.56,77.18


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3161393.0,21
AR,1333534.0,10
XR,1281104.0,13
X40,984773.0,14
X40N,882882.0,9
OX40N,737184.0,10
XY25,352568.0,6
SR,281637.0,2
MH,81233.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3640796.0,25
M-SC,1449611.0,15
X-MC,1441035.0,16
X-LC,808000.0,11
X-SC,792769.0,8
H-MC,400145.0,3
L-SC,243157.0,3
M-LC,106813.0,1
H-LC,79135.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1255006.0      6
           AR         910974.0      5
M-SC       XY24       819720.0      6
H-SC       XR         782574.0      7
X-MC       X40        487833.0      7
           XY24       414117.0      3
           X40N       352874.0      4
H-SC       OX40N      329372.0      4
M-SC       OX40N      324109.0      5
X-LC       X40        317807.0      5
X-SC       X40N       315148.0      3
           XY24       298488.0      3
H-SC       SR         281637.0      2
H-MC       AR         215758.0      2
X-LC       X40N       214860.0      2
           XY24       189675.0      2
X-MC       XY25       186211.0      2
H-MC       XY24       184387.0      1
X-SC       X40        179133.0      2
M-SC       XR         178115.0      2
L-SC       XR         159454.0      2
M-SC       AR         127667.0      2
M-LC       XR         106813.0      1
X-LC       XY25        85658.0      2
L-SC       OX40N       83703.0      1
H-SC       MH          81233.0      1
H-LC       AR          79135.0      1
L-MC       XR          54148.0      1
M-MC       XY25        50107.0      1
L-LC       XY25        30592.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
